운수업조사
-------------
https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1I100001&conn_path=I2

In [1]:
from tensorflow import keras

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [18]:
pd_data = pd.read_csv('시도별_산업별_총괄.csv', encoding='cp949')
print(pd_data[1:11])
# print(pd_data.shape)

# 2007년 ~ 2018년도(2010년 없음)
# insert(10부터 9씩증가, 'year' + 2017부터 1씩증가, pd.Series([2007부터 1씩증가]*69))
pd_data.insert(10, 'year2007', pd.Series([2007]*69))
pd_data.insert(19, 'year2008', pd.Series([2008]*69))
pd_data.insert(28, 'year2009', pd.Series([2009]*69))

pd_data.insert(37, 'year2011', pd.Series([2011]*69))
pd_data.insert(46, 'year2012', pd.Series([2012]*69))
pd_data.insert(55, 'year2013', pd.Series([2013]*69))
pd_data.insert(64, 'year2014', pd.Series([2014]*69))
pd_data.insert(73, 'year2015', pd.Series([2015]*69))
pd_data.insert(82, 'year2016', pd.Series([2016]*69))
pd_data.insert(91, 'year2017', pd.Series([2017]*69))
pd_data.insert(100, 'year2018', pd.Series([2018]*69))

print(pd_data.iloc[:,97:101])

      시도별               산업별   2007  2007.1   2007.2  2007.3    2007.4  \
1   서울특별시  육상운송 및 파이프라인 운송업  88381  269769  4656584  196733  13956845   
2   서울특별시            수상 운송업    139   16632   852384    1472  31864040   
3   서울특별시            항공 운송업      7   21378  1151139     224  12508641   
4   서울특별시    창고 및 운송관련 서비스업   4417   61180  1498344     550   4928076   
5   부산광역시  육상운송 및 파이프라인 운송업  25200   85420  1629907   63936   5062509   
6   부산광역시            수상 운송업     87    3036    87602     295    556389   
7   부산광역시            항공 운송업      -       -        -       -         -   
8   부산광역시    창고 및 운송관련 서비스업   3814   15296   350200     346   1517261   
9   대구광역시  육상운송 및 파이프라인 운송업  19015   55185   654504   42769   2089917   
10  대구광역시            수상 운송업      -       -        -       -         -   

      2007.5   2007.6    2007.7  ...    2017.6    2017.7   2018  2018.1  \
1   11775469  8766262  17805625  ...  11416594  22246183  87865  264663   
2   29945491  5326603  17204269  ...   3444312